In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyreadstat
import seaborn as sns
from scipy import stats
import seaborn as sns

import shutil
import glob
import sys
import os

/home/marijatochadse/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/marijatochadse/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
def calc_p_values(df):
    
    df["p_value"] = np.nan
    if 'io' in df:
        grp_order = ["io", "technique", "model",  "inp", "out", "repetition"] #this should be grouped by repetition run as well 
                                                               #(to have 1000 perm vals instead of 10.000)
    else:
        grp_order = ["technique", "model",  "inp", "out", "repetition"]
    if 'i_type' in df and len(df['i_type'].unique())>1:
        grp_order.insert(0, 'i_type')       
    groups = df.dropna(subset=["val_metric", "permuted_test_score"]).groupby(grp_order)
            
    for i, (g, rows) in enumerate(groups):
        
        p_vals = []
        rows = rows.filter(items=['val_metric', 'permuted_test_score'])
        testrows = np.array(rows['permuted_test_score'])
         
        perm_vals = []
        for j in range(rows.shape[0]):
            rowstr = testrows[j]
            fold_perm = eval(rowstr)
            perm_vals = perm_vals + fold_perm
        
        for _, r in rows.iterrows():
            p_scores = np.array(perm_vals)

            true_score = r['val_metric']
            C = np.sum(p_scores >= r['val_metric'])
            pi = (C+1)/(len(p_scores)+1)
            p_vals.extend([pi])
            

        df.loc[(df[grp_order]==g).all(axis=1), ["p_value"]] = np.mean(p_vals)  
        
    return df

## Response - Structural MRI

In [8]:
results_path = '/home/marijatochadse/2_scripts/MLpipeline_Marija/results/classification_Marija_MRI/MRI_OUTResponse/20240106-1240/run.csv'
df = pd.read_csv(results_path, sep=",")

In [9]:
df.shape

(280, 24)

In [10]:
df.head()

,task_type,model,analysis,repetition,trial,inp,out,technique,confs,n_samples,n_samples_cc,model_metric,innerval_metric,m__params,train_metric,val_metric,val_ids,train_ids,val_lbls,val_preds,permuted_test_score,permuted_roc_auc,other_metrics,runtime
0,classification,GB,classification_baseline_Response_GB,7,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.541111,{'model_GB__learning_rate': 0.25},0.993281,0.513889,"[10, 11, 13, 14, 23, 26, 30, 41, 54, 57, 70, 7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 15, 16, 17,...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[[0.43689998984336853, 0.5630999803543091], [0...","[0.5138888888888888, 0.5, 0.6388888888888888, ...","[0.3611111111111111, 0.4444444444444444, 0.555...",{'roc_auc': 0.4722222222222222},604
1,classification,GB,classification_baseline_Response_GB,4,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.537778,{'model_GB__learning_rate': 0.1},0.996591,0.513889,"[0, 3, 13, 18, 22, 34, 39, 44, 46, 48, 53, 76,...","[1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 1...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[[0.16850000619888306, 0.8314999938011169], [0...","[0.5138888888888888, 0.4444444444444444, 0.625...","[0.41666666666666663, 0.41666666666666663, 0.6...",{'roc_auc': 0.4722222222222222},624
2,classification,GB,classification_baseline_Response_GB,8,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.542222,{'model_GB__learning_rate': 0.1},0.995553,0.458333,"[8, 9, 17, 19, 34, 38, 44, 49, 54, 62, 65, 87,...","[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 1...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...","[[0.054499998688697815, 0.9455000162124634], [...","[0.3888888888888889, 0.4027777777777778, 0.583...","[0.4722222222222222, 0.5277777777777778, 0.527...",{'roc_auc': 0.5},547
3,classification,GB,classification_baseline_Response_GB,0,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.531111,{'model_GB__learning_rate': 0.25},0.995504,0.388889,"[12, 17, 21, 30, 39, 45, 55, 59, 62, 65, 73, 7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14,...","[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","[[0.11490000039339066, 0.8851000070571899], [0...","[0.5694444444444444, 0.3333333333333333, 0.5, ...","[0.8055555555555556, 0.2222222222222222, 0.777...",{'roc_auc': 0.5},625
4,classification,GB,classification_baseline_Response_GB,3,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.503333,{'model_GB__learning_rate': 0.05},0.997727,0.569444,"[1, 5, 8, 11, 13, 15, 45, 48, 50, 61, 63, 77, 84]","[0, 2, 3, 4, 6, 7, 9, 10, 12, 14, 16, 17, 18, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ...","[[0.5138000249862671, 0.4862000048160553], [0....","[0.3333333333333333, 0.3888888888888889, 0.277...","[0.2222222222222222, 0.19444444444444442, 0.19...",{'roc_auc': 0.6666666666666666},495


In [11]:
df = calc_p_values(df)
df.head()

,task_type,model,analysis,repetition,trial,inp,out,technique,confs,n_samples,n_samples_cc,model_metric,innerval_metric,m__params,train_metric,val_metric,val_ids,train_ids,val_lbls,val_preds,permuted_test_score,permuted_roc_auc,other_metrics,runtime,p_value
0,classification,GB,classification_baseline_Response_GB,7,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.541111,{'model_GB__learning_rate': 0.25},0.993281,0.513889,"[10, 11, 13, 14, 23, 26, 30, 41, 54, 57, 70, 7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 15, 16, 17,...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[[0.43689998984336853, 0.5630999803543091], [0...","[0.5138888888888888, 0.5, 0.6388888888888888, ...","[0.3611111111111111, 0.4444444444444444, 0.555...",{'roc_auc': 0.4722222222222222},604,0.565829
1,classification,GB,classification_baseline_Response_GB,4,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.537778,{'model_GB__learning_rate': 0.1},0.996591,0.513889,"[0, 3, 13, 18, 22, 34, 39, 44, 46, 48, 53, 76,...","[1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 1...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[[0.16850000619888306, 0.8314999938011169], [0...","[0.5138888888888888, 0.4444444444444444, 0.625...","[0.41666666666666663, 0.41666666666666663, 0.6...",{'roc_auc': 0.4722222222222222},624,0.603302
2,classification,GB,classification_baseline_Response_GB,8,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.542222,{'model_GB__learning_rate': 0.1},0.995553,0.458333,"[8, 9, 17, 19, 34, 38, 44, 49, 54, 62, 65, 87,...","[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 1...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...","[[0.054499998688697815, 0.9455000162124634], [...","[0.3888888888888889, 0.4027777777777778, 0.583...","[0.4722222222222222, 0.5277777777777778, 0.527...",{'roc_auc': 0.5},547,0.569706
3,classification,GB,classification_baseline_Response_GB,0,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.531111,{'model_GB__learning_rate': 0.25},0.995504,0.388889,"[12, 17, 21, 30, 39, 45, 55, 59, 62, 65, 73, 7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14,...","[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","[[0.11490000039339066, 0.8851000070571899], [0...","[0.5694444444444444, 0.3333333333333333, 0.5, ...","[0.8055555555555556, 0.2222222222222222, 0.777...",{'roc_auc': 0.5},625,0.708543
4,classification,GB,classification_baseline_Response_GB,3,0,X,Response,baseline,[''],90,90,balanced_accuracy,0.503333,{'model_GB__learning_rate': 0.05},0.997727,0.569444,"[1, 5, 8, 11, 13, 15, 45, 48, 50, 61, 63, 77, 84]","[0, 2, 3, 4, 6, 7, 9, 10, 12, 14, 16, 17, 18, ...","[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ...","[[0.5138000249862671, 0.4862000048160553], [0....","[0.3333333333333333, 0.3888888888888889, 0.277...","[0.2222222222222222, 0.19444444444444442, 0.19...",{'roc_auc': 0.6666666666666666},495,0.512132


In [12]:
group_pvals = df[["technique", "model", "p_value"]]
group_pvals_mean = group_pvals.groupby(["technique", "model"])['p_value'].mean().reset_index()
group_pvals_mean

,technique,model,p_value
0,baseline,GB,0.581479
1,baseline,LR(c),0.571859
2,baseline,SVM-lin,0.655233
3,baseline,SVM-rbf,0.388973


In [13]:
group_valmetric = df[["technique", "model", "val_metric"]]
group_valmetric_mean = group_valmetric.groupby(["technique", "model"])['val_metric'].mean().reset_index()
group_valmetric_mean

,technique,model,val_metric
0,baseline,GB,0.479246
1,baseline,LR(c),0.510099
2,baseline,SVM-lin,0.494028
3,baseline,SVM-rbf,0.570516


## Response - fMRI graph

In [25]:
results_path = '/home/marijatochadse/2_scripts/MLpipeline_Marija/results/classification_Marija/graph_fMRI_OUTResponse/20240411-1553/run.csv'
df = pd.read_csv(results_path, sep=",")

In [26]:
df.shape

(280, 24)

In [29]:
df = calc_p_values(df)
df.head()

,task_type,model,analysis,repetition,trial,inp,out,technique,confs,n_samples,n_samples_cc,model_metric,innerval_metric,m__params,train_metric,val_metric,val_ids,train_ids,val_lbls,val_preds,permuted_test_score,permuted_roc_auc,other_metrics,runtime,p_value
0,classification,GB,classification_baseline_Response_GB,0,0,X,Response,baseline,[''],80,80,balanced_accuracy,0.686111,{'model_GB__learning_rate': 0.05},0.928372,0.5625,"[0, 15, 36, 38, 45, 51, 53, 56, 60, 69, 70, 73]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, ...","[[0.2206999957561493, 0.7792999744415283], [0....","[0.4375, 0.5625, 0.4375, 0.4375, 0.5, 0.5625, ...","[0.4375, 0.515625, 0.53125, 0.53125, 0.65625, ...",{'roc_auc': 0.671875},192,0.292175
1,classification,GB,classification_baseline_Response_GB,8,0,X,Response,baseline,[''],80,80,balanced_accuracy,0.613056,{'model_GB__learning_rate': 0.01},0.925830,0.4375,"[6, 14, 18, 26, 27, 40, 42, 46, 53, 60, 71, 79]","[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 15...","[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...","[[0.17470000684261322, 0.8252999782562256], [0...","[0.5625, 0.5, 0.5625, 0.4375, 0.4375, 0.4375, ...","[0.34375, 0.546875, 0.4375, 0.34375, 0.421875,...",{'roc_auc': 0.34375},201,0.444221
2,classification,GB,classification_baseline_Response_GB,9,0,X,Response,baseline,[''],80,80,balanced_accuracy,0.542500,{'model_GB__learning_rate': 0.25},0.919601,0.6875,"[2, 13, 18, 25, 26, 39, 42, 44, 60, 68, 73, 74]","[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15...","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[[0.061900001019239426, 0.9380999803543091], [...","[0.375, 0.4375, 0.3125, 0.3125, 0.375, 0.625, ...","[0.5625, 0.5, 0.125, 0.28125, 0.375, 0.625, 0....",{'roc_auc': 0.8125},167,0.402297
3,classification,GB,classification_baseline_Response_GB,3,0,X,Response,baseline,[''],80,80,balanced_accuracy,0.553333,{'model_GB__learning_rate': 0.1},0.927815,0.6875,"[7, 8, 9, 14, 15, 31, 45, 52, 54, 67, 71, 76]","[0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 16, 17, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[[0.21619999408721924, 0.7838000059127808], [0...","[0.8125, 0.4375, 0.5, 0.5625, 0.4375, 0.5625, ...","[0.75, 0.40625, 0.375, 0.46875, 0.34375, 0.687...",{'roc_auc': 0.875},203,0.251831
4,classification,GB,classification_baseline_Response_GB,6,0,X,Response,baseline,[''],80,80,balanced_accuracy,0.535833,{'model_GB__learning_rate': 0.05},0.930830,0.5000,"[2, 9, 10, 16, 25, 34, 36, 47, 56, 59, 68, 78]","[0, 1, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 1...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[[0.11749999970197678, 0.8824999928474426], [0...","[0.625, 0.4375, 0.4375, 0.375, 0.625, 0.4375, ...","[0.484375, 0.25, 0.09375, 0.4375, 0.65625, 0.4...",{'roc_auc': 0.5},187,0.420388


In [30]:
group_pvals = df[["technique", "model", "p_value"]]
group_pvals_mean = group_pvals.groupby(["technique", "model"])['p_value'].mean().reset_index()
group_pvals_mean

,technique,model,p_value
0,baseline,GB,0.361809
1,baseline,LR(c),0.680043
2,baseline,SVM-lin,0.570510
3,baseline,SVM-rbf,0.673941


In [31]:
group_valmetric = df[["technique", "model", "val_metric"]]
group_valmetric_mean = group_valmetric.groupby(["technique", "model"])['val_metric'].mean().reset_index()
group_valmetric_mean

,technique,model,val_metric
0,baseline,GB,0.580255
1,baseline,LR(c),0.476658
2,baseline,SVM-lin,0.506352
3,baseline,SVM-rbf,0.452653


## Remission - TAB all

In [14]:
results_path = '/home/marijatochadse/2_scripts/MLpipeline_Marija/results/classification_Marija_CONFSSex/TAB_OUTRemission_CONFSSex/20240130-1045/run.csv'
df = pd.read_csv(results_path, sep=",")
df.shape

(832, 24)

In [15]:
df = df[df["inp"]=="X"]
df = df[df["out"]=="Remission"]
df.shape

(277, 24)

In [16]:
df = calc_p_values(df)
df.head()

,task_type,model,analysis,repetition,trial,inp,out,technique,confs,n_samples,n_samples_cc,model_metric,innerval_metric,m__params,train_metric,val_metric,val_ids,train_ids,val_lbls,val_preds,permuted_test_score,permuted_roc_auc,other_metrics,runtime,p_value
555,classification,GB,classification_baseline_Remission_GB,7,0,X,Remission,cb,['Geschlecht'],119,141,balanced_accuracy,0.733636,{'model_GB__learning_rate': 0.25},0.955869,0.708333,"[6, 11, 15, 17, 26, 27, 30, 44, 57, 59, 72, 80...","[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 16...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[[0.95660001039505, 0.04340000078082085], [0.4...","[0.5138888888888888, 0.4097222222222222, 0.534...","[0.4305555555555556, 0.26388888888888884, 0.51...",{'roc_auc': 0.7638888888888888},207,0.143431
556,classification,GB,classification_baseline_Remission_GB,5,0,X,Remission,cb,['Geschlecht'],119,135,balanced_accuracy,0.705455,{'model_GB__learning_rate': 0.1},0.954055,0.701389,"[0, 9, 27, 35, 40, 44, 49, 54, 56, 63, 85, 91,...","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[[0.42399999499320984, 0.5759999752044678], [0...","[0.4652777777777778, 0.5277777777777778, 0.409...","[0.6388888888888888, 0.41666666666666663, 0.61...",{'roc_auc': 0.7222222222222222},205,0.071788
557,classification,GB,classification_baseline_Remission_GB,2,0,X,Remission,cb,['Geschlecht'],119,135,balanced_accuracy,0.655455,{'model_GB__learning_rate': 0.1},0.953567,0.770833,"[13, 23, 27, 30, 53, 60, 62, 69, 74, 77, 79, 9...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14,...","[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...","[[0.22849999368190765, 0.7714999914169312], [0...","[0.5208333333333333, 0.41666666666666663, 0.58...","[0.5, 0.4861111111111111, 0.625, 0.73611111111...",{'roc_auc': 0.8472222222222222},151,0.155922
558,classification,GB,classification_baseline_Remission_GB,3,0,X,Remission,cb,['Geschlecht'],119,139,balanced_accuracy,0.660000,{'model_GB__learning_rate': 0.05},0.958338,0.826389,"[5, 9, 12, 24, 27, 30, 32, 35, 52, 54, 67, 70,...","[0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 13, 14, 15, 1...","[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[[0.9710000157356262, 0.028999999165534973], [...","[0.3472222222222222, 0.29166666666666663, 0.45...","[0.25, 0.2361111111111111, 0.38888888888888884...",{'roc_auc': 0.8472222222222222},211,0.190955
559,classification,GB,classification_baseline_Remission_GB,8,0,X,Remission,cb,['Geschlecht'],119,137,balanced_accuracy,0.686364,{'model_GB__learning_rate': 0.05},0.965686,0.701389,"[8, 10, 21, 22, 26, 31, 37, 48, 50, 55, 56, 70...","[0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[[0.7703999876976013, 0.2295999974012375], [0....","[0.4236111111111111, 0.29166666666666663, 0.40...","[0.625, 0.15277777777777776, 0.402777777777777...",{'roc_auc': 0.8888888888888888},199,0.134386


In [17]:
group_pvals = df[["technique", "model",  "p_value"]]
group_pvals_mean = group_pvals.groupby(["technique", "model"])['p_value'].mean().reset_index()
group_pvals_mean

,technique,model,p_value
0,cb,GB,0.154716
1,cb,LR(c),0.227504
2,cb,SVM-lin,0.220455
3,cb,SVM-rbf,0.207528


In [18]:
group_valmetric = df[["technique", "model", "val_metric"]]
group_valmetric_mean = group_valmetric.groupby(["technique", "model"])['val_metric'].mean().reset_index()
group_valmetric_mean

,technique,model,val_metric
0,cb,GB,0.679067
1,cb,LR(c),0.644424
2,cb,SVM-lin,0.654489
3,cb,SVM-rbf,0.655998


## Remission - TAB no OCD

In [70]:
results_path = '/home/marijatochadse/2_scripts/MLpipeline_Marija/results/classification_Marija_CONFSSex/TAB_noclin_OUTRemission_CONFSSex/20240409-1437/run.csv'
df = pd.read_csv(results_path, sep=",")
df.shape

(832, 24)

In [71]:
df = df[df["inp"]=="X"]
df = df[df["out"]=="Remission"]
df.shape

(277, 24)

In [72]:
df = calc_p_values(df)
df.head()

,task_type,model,analysis,repetition,trial,inp,out,technique,confs,n_samples,n_samples_cc,model_metric,innerval_metric,m__params,train_metric,val_metric,val_ids,train_ids,val_lbls,val_preds,permuted_test_score,permuted_roc_auc,other_metrics,runtime,p_value
555,classification,GB,classification_baseline_Remission_GB,5,0,X,Remission,cb,['Geschlecht'],119,135,balanced_accuracy,0.618182,{'model_GB__learning_rate': 0.05},0.939421,0.590278,"[0, 9, 27, 35, 40, 44, 49, 54, 56, 63, 85, 91,...","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[[0.5792999863624573, 0.4207000136375427], [0....","[0.4097222222222222, 0.6388888888888888, 0.590...","[0.5555555555555556, 0.5694444444444444, 0.569...",{'roc_auc': 0.5138888888888888},205,0.250826
556,classification,GB,classification_baseline_Remission_GB,2,0,X,Remission,cb,['Geschlecht'],119,135,balanced_accuracy,0.595455,{'model_GB__learning_rate': 0.25},0.942546,0.645833,"[13, 23, 27, 30, 53, 60, 62, 69, 74, 77, 79, 9...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14,...","[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...","[[0.32190001010894775, 0.6780999898910522], [0...","[0.5277777777777778, 0.5347222222222222, 0.652...","[0.45833333333333337, 0.41666666666666663, 0.7...",{'roc_auc': 0.7083333333333334},201,0.263604
557,classification,GB,classification_baseline_Remission_GB,7,0,X,Remission,cb,['Geschlecht'],119,141,balanced_accuracy,0.606364,{'model_GB__learning_rate': 0.05},0.943034,0.708333,"[6, 11, 15, 17, 26, 27, 30, 44, 57, 59, 72, 80...","[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 16...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[[0.8884000182151794, 0.11159999668598175], [0...","[0.47916666666666663, 0.47916666666666663, 0.5...","[0.5277777777777777, 0.3611111111111111, 0.486...",{'roc_auc': 0.7222222222222222},198,0.236037
558,classification,GB,classification_baseline_Remission_GB,8,0,X,Remission,cb,['Geschlecht'],119,137,balanced_accuracy,0.520000,{'model_GB__learning_rate': 0.25},0.946631,0.645833,"[8, 10, 21, 22, 26, 31, 37, 48, 50, 55, 56, 70...","[0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15...","[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[[0.7876999974250793, 0.21230000257492065], [0...","[0.6388888888888888, 0.6388888888888888, 0.763...","[0.7083333333333333, 0.7083333333333333, 0.722...",{'roc_auc': 0.6527777777777778},200,0.246949
559,classification,GB,classification_baseline_Remission_GB,3,0,X,Remission,cb,['Geschlecht'],119,139,balanced_accuracy,0.656364,{'model_GB__learning_rate': 0.1},0.954756,0.590278,"[5, 9, 12, 24, 27, 30, 32, 35, 52, 54, 67, 70,...","[0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 13, 14, 15, 1...","[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[[0.9595000147819519, 0.04050000011920929], [0...","[0.4722222222222222, 0.4722222222222222, 0.638...","[0.5694444444444444, 0.6111111111111112, 0.694...",{'roc_auc': 0.5555555555555556},204,0.421536


In [73]:
group_pvals = df[["technique", "model",  "p_value"]]
group_pvals_mean = group_pvals.groupby(["technique", "model"])['p_value'].mean().reset_index()
group_pvals_mean

,technique,model,p_value
0,cb,GB,0.300876
1,cb,LR(c),0.279757
2,cb,SVM-lin,0.260213
3,cb,SVM-rbf,0.330804


In [74]:
group_valmetric = df[["technique", "model", "val_metric"]]
group_valmetric_mean = group_valmetric.groupby(["technique", "model"])['val_metric'].mean().reset_index()
group_valmetric_mean

,technique,model,val_metric
0,cb,GB,0.589782
1,cb,LR(c),0.615338
2,cb,SVM-lin,0.621779
3,cb,SVM-rbf,0.589271


## Remission - multimodal

In [35]:
results_path = '/home/marijatochadse/2_scripts/MLpipeline_Marija/results/classification_Marija_MRI_Remission/TABnoclin_sMRI_OUTRemission_CONFSSex/20240410-1124/run.csv'
df = pd.read_csv(results_path, sep=",")
df.shape

(832, 24)

In [36]:
df = df[df["inp"]=="X"]
df = df[df["out"]=="Remission"]
df.shape

(277, 24)

In [37]:
df = calc_p_values(df)
df.head()

,task_type,model,analysis,repetition,trial,inp,out,technique,confs,n_samples,n_samples_cc,model_metric,innerval_metric,m__params,train_metric,val_metric,val_ids,train_ids,val_lbls,val_preds,permuted_test_score,permuted_roc_auc,other_metrics,runtime,p_value
555,classification,GB,classification_baseline_Remission_GB,9,0,X,Remission,cb,['Geschlecht'],90,103,balanced_accuracy,0.601786,{'model_GB__learning_rate': 0.05},0.954327,0.833333,"[0, 7, 17, 31, 32, 36, 44, 50, 56, 58, 79, 82,...","[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...","[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[[0.10719999670982361, 0.892799973487854], [0....","[0.4642857142857143, 0.4642857142857143, 0.523...","[0.40476190476190477, 0.5476190476190477, 0.52...",{'roc_auc': 0.9047619047619049},186,0.170567
556,classification,GB,classification_baseline_Remission_GB,6,0,X,Remission,cb,['Geschlecht'],90,107,balanced_accuracy,0.585714,{'model_GB__learning_rate': 0.25},0.968606,0.761905,"[8, 17, 22, 24, 44, 47, 48, 52, 53, 60, 72, 79...","[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14...","[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...","[[0.1590999960899353, 0.8409000039100647], [0....","[0.5476190476190476, 0.7023809523809523, 0.690...","[0.5714285714285714, 0.6904761904761905, 0.619...",{'roc_auc': 0.6904761904761904},193,0.199569
557,classification,GB,classification_baseline_Remission_GB,4,0,X,Remission,cb,['Geschlecht'],90,103,balanced_accuracy,0.598214,{'model_GB__learning_rate': 0.25},0.951108,0.595238,"[3, 6, 7, 13, 31, 32, 33, 40, 44, 58, 62, 76, 81]","[0, 1, 2, 4, 5, 8, 9, 10, 11, 12, 14, 15, 16, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...","[[0.8924999833106995, 0.10750000178813934], [0...","[0.38095238095238093, 0.4642857142857143, 0.46...","[0.4761904761904762, 0.2857142857142857, 0.547...",{'roc_auc': 0.6904761904761904},193,0.417373
558,classification,GB,classification_baseline_Remission_GB,0,0,X,Remission,cb,['Geschlecht'],90,107,balanced_accuracy,0.607143,{'model_GB__learning_rate': 0.05},0.966530,0.464286,"[2, 13, 29, 38, 45, 51, 52, 60, 66, 76, 77, 80...","[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15...","[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...","[[0.5063999891281128, 0.4936000108718872], [0....","[0.5357142857142857, 0.5476190476190476, 0.559...","[0.5, 0.5714285714285714, 0.6190476190476191, ...",{'roc_auc': 0.47619047619047616},164,0.201005
559,classification,GB,classification_baseline_Remission_GB,1,0,X,Remission,cb,['Geschlecht'],90,99,balanced_accuracy,0.491071,{'model_GB__learning_rate': 0.25},0.939847,0.464286,"[13, 20, 23, 27, 45, 49, 50, 63, 64, 66, 70, 7...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14,...","[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...","[[0.867900013923645, 0.13210000097751617], [0....","[0.6190476190476191, 0.47619047619047616, 0.22...","[0.6666666666666667, 0.5238095238095238, 0.214...",{'roc_auc': 0.5476190476190477},186,0.340704


In [38]:
group_pvals = df[["technique", "model",  "p_value"]]
group_pvals_mean = group_pvals.groupby(["technique", "model"])['p_value'].mean().reset_index()
group_pvals_mean

,technique,model,p_value
0,cb,GB,0.290912
1,cb,LR(c),0.321558
2,cb,SVM-lin,0.254305
3,cb,SVM-rbf,0.311407


In [39]:
group_valmetric = df[["technique", "model", "val_metric"]]
group_valmetric_mean = group_valmetric.groupby(["technique", "model"])['val_metric'].mean().reset_index()
group_valmetric_mean

,technique,model,val_metric
0,cb,GB,0.621939
1,cb,LR(c),0.602312
2,cb,SVM-lin,0.636991
3,cb,SVM-rbf,0.601449
